In [25]:
import time
import os
from tqdm import tqdm

import pandas as pd
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

# 忽略烦人的红色提示
import warnings
warnings.filterwarnings("ignore")

# 获取计算硬件
# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cpu


In [26]:
from torchvision import transforms

# 训练集图像预处理：缩放裁剪、图像增强、转 Tensor、归一化
train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])

# 测试集图像预处理-RCTN：缩放、裁剪、转 Tensor、归一化
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

In [27]:
# 数据集文件夹路径
dataset_dir = 'machi5_split'

In [28]:
train_path = os.path.join(dataset_dir, 'train')
test_path = os.path.join(dataset_dir, 'val')
print('训练集路径', train_path)
print('测试集路径', test_path)

from torchvision import datasets
# 载入训练集
train_dataset = datasets.ImageFolder(train_path, train_transform)
# 载入测试集
test_dataset = datasets.ImageFolder(test_path, test_transform)

print('训练集图像数量', len(train_dataset))
print('类别个数', len(train_dataset.classes))
print('各类别名称', train_dataset.classes)
print('测试集图像数量', len(test_dataset))
print('类别个数', len(test_dataset.classes))
print('各类别名称', test_dataset.classes)

训练集路径 machi5_split\train
测试集路径 machi5_split\val
训练集图像数量 1103
类别个数 5
各类别名称 ['Lv1', 'Lv2', 'lv3', 'lv4', 'lv5']
测试集图像数量 56
类别个数 5
各类别名称 ['Lv1', 'Lv2', 'lv3', 'lv4', 'lv5']


In [29]:
# 各类别名称
class_names = train_dataset.classes
n_class = len(class_names)
# 映射关系：类别 到 索引号
train_dataset.class_to_idx
# 映射关系：索引号 到 类别
idx_to_labels = {y:x for x,y in train_dataset.class_to_idx.items()}

In [30]:
idx_to_labels

{0: 'Lv1', 1: 'Lv2', 2: 'lv3', 3: 'lv4', 4: 'lv5'}

In [31]:
# 保存为本地的 npy 文件
np.save('idx_to_labels.npy', idx_to_labels)
np.save('labels_to_idx.npy', train_dataset.class_to_idx)

In [32]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32

# 训练集的数据加载器
train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=4
                         )

# 测试集的数据加载器
test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=4
                        )

In [33]:
from torchvision import models
import torch.optim as optim
from torch.optim import lr_scheduler

In [34]:
model = models.resnet18(pretrained=True) # 载入预训练模型

model.fc = nn.Linear(model.fc.in_features, n_class)

optimizer = optim.Adam(model.parameters())

In [35]:
# model = models.resnet18(pretrained=False) # 只载入模型结构，不载入预训练权重参数

# model.fc = nn.Linear(model.fc.in_features, n_class)

# optimizer = optim.Adam(model.parameters())

In [36]:
model = model.to(device)

# 交叉熵损失函数
criterion = nn.CrossEntropyLoss() 

# 训练轮次 Epoch
EPOCHS = 100

# 学习率降低策略
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [37]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [38]:
def train_one_batch(images, labels):
    '''
    运行一个 batch 的训练，返回当前 batch 的训练日志
    '''
    
    # 获得一个 batch 的数据和标注
    images = images.to(device)
    labels = labels.to(device)
    
    outputs = model(images) # 输入模型，执行前向预测
    loss = criterion(outputs, labels) # 计算当前 batch 中，每个样本的平均交叉熵损失函数值
    
    # 优化更新权重
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # 获取当前 batch 的标签类别和预测类别
    _, preds = torch.max(outputs, 1) # 获得当前 batch 所有图像的预测类别
    preds = preds.cpu().numpy()
    loss = loss.detach().cpu().numpy()
    outputs = outputs.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    
    log_train = {}
    log_train['epoch'] = epoch
    log_train['batch'] = batch_idx
    # 计算分类评估指标
    log_train['train_loss'] = loss
    log_train['train_accuracy'] = accuracy_score(labels, preds)
    # log_train['train_precision'] = precision_score(labels, preds, average='macro')
    # log_train['train_recall'] = recall_score(labels, preds, average='macro')
    # log_train['train_f1-score'] = f1_score(labels, preds, average='macro')
    
    return log_train

In [39]:
def evaluate_testset():
    '''
    在整个测试集上评估，返回分类评估指标日志
    '''

    loss_list = []
    labels_list = []
    preds_list = []
    
    with torch.no_grad():
        for images, labels in test_loader: # 生成一个 batch 的数据和标注
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images) # 输入模型，执行前向预测

            # 获取整个测试集的标签类别和预测类别
            _, preds = torch.max(outputs, 1) # 获得当前 batch 所有图像的预测类别
            preds = preds.cpu().numpy()
            loss = criterion(outputs, labels) # 由 logit，计算当前 batch 中，每个样本的平均交叉熵损失函数值
            loss = loss.detach().cpu().numpy()
            outputs = outputs.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()

            loss_list.append(loss)
            labels_list.extend(labels)
            preds_list.extend(preds)
        
    log_test = {}
    log_test['epoch'] = epoch
    
    # 计算分类评估指标
    log_test['test_loss'] = np.mean(loss_list)
    log_test['test_accuracy'] = accuracy_score(labels_list, preds_list)
    log_test['test_precision'] = precision_score(labels_list, preds_list, average='macro')
    log_test['test_recall'] = recall_score(labels_list, preds_list, average='macro')
    log_test['test_f1-score'] = f1_score(labels_list, preds_list, average='macro')
    
    return log_test

In [40]:
epoch = 0
batch_idx = 0
best_test_accuracy = 0

In [41]:
# 训练日志-训练集
df_train_log = pd.DataFrame()
log_train = {}
log_train['epoch'] = 0
log_train['batch'] = 0
images, labels = next(iter(train_loader))
log_train.update(train_one_batch(images, labels))
df_train_log = pd.concat([df_train_log, pd.DataFrame(log_train, index=[0])], ignore_index=True)

In [42]:
df_train_log

,epoch,batch,train_loss,train_accuracy
0,0,0,1.731076,0.21875


In [43]:
# 训练日志-测试集
df_test_log = pd.DataFrame()
log_test = {}
log_test['epoch'] = 0
log_test.update(evaluate_testset())
# 修改后的代码
df_test_log = pd.concat([df_test_log, pd.DataFrame(log_test, index=[0])], ignore_index=True)

In [44]:
df_test_log

,epoch,test_loss,test_accuracy,test_precision,test_recall,test_f1-score
0,0,1.540832,0.482143,0.377177,0.395343,0.380673


## 登录wandb

1.安装 wandb：pip install wandb

2.登录 wandb：在命令行中运行wandb login

3.按提示复制粘贴API Key至命令行中

## 创建wandb可视化项目

In [45]:
import wandb

wandb.init(project='machi5', name=time.strftime('%m%d%H%M%S'))

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x0000016BD987DA30>> (for post_run_cell), with arguments args (<ExecutionResult object at 16bd46f3d90, execution_count=45 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 16bd46f3790, raw_cell="import wandb

wandb.init(project='machi5', name=ti.." store_history=True silent=False shell_futures=True cell_id=eb211f1a-64fc-497a-abf5-5639618a3784> result=<wandb.sdk.wandb_run.Run object at 0x0000016BD47A2D00>>,),kwargs {}:


TypeError: _pause_backend() takes 1 positional argument but 2 were given

In [46]:
for epoch in range(1, EPOCHS+1):
    
    print(f'Epoch {epoch}/{EPOCHS}')
    
    ## 训练阶段
    model.train()
    for images, labels in tqdm(train_loader): # 获得一个 batch 的数据和标注
        batch_idx += 1
        log_train = train_one_batch(images, labels)
        # 修改后的代码
        df_train_log = pd.concat([df_train_log, pd.DataFrame(log_train, index=[0])], ignore_index=True)
        wandb.log(log_train)
    lr_scheduler.step()

    ## 测试阶段
    model.eval()
    log_test = evaluate_testset()
    df_test_log = pd.concat([df_test_log, pd.DataFrame(log_test, index=[0])], ignore_index=True)
    wandb.log(log_test)
    
    # 保存最新的最佳模型文件
    if log_test['test_accuracy'] > best_test_accuracy: 
        # 删除旧的最佳模型文件(如有)
        old_best_checkpoint_path = 'checkpoint/best-{:.3f}.pth'.format(best_test_accuracy)
        if os.path.exists(old_best_checkpoint_path):
            os.remove(old_best_checkpoint_path)
        # 保存新的最佳模型文件
        best_test_accuracy = log_test['test_accuracy']
        new_best_checkpoint_path = 'checkpoint/best-{:.3f}.pth'.format(log_test['test_accuracy'])
        torch.save(model, new_best_checkpoint_path)
        print('保存新的最佳模型', 'checkpoint/best-{:.3f}.pth'.format(best_test_accuracy))
        # best_test_accuracy = log_test['test_accuracy']

df_train_log.to_csv('trainLog.csv', index=False)
df_test_log.to_csv('testLog.csv', index=False)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x0000016BD987DA30>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 16bd9872880, raw_cell="for epoch in range(1, EPOCHS+1):
    
    print(f'.." store_history=True silent=False shell_futures=True cell_id=9000ce4e-7134-4c56-b95b-926cb5194b18>,),kwargs {}:


TypeError: _resume_backend() takes 1 positional argument but 2 were given

Epoch 1/100


100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [01:04<00:00,  1.85s/it]


保存新的最佳模型 checkpoint/best-0.554.pth
Epoch 2/100


...

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [01:03<00:00,  1.83s/it]


Epoch 100/100


100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [01:04<00:00,  1.83s/it]


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x0000016BD987DA30>> (for post_run_cell), with arguments args (<ExecutionResult object at 16bd9862ac0, execution_count=46 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 16bd9872880, raw_cell="for epoch in range(1, EPOCHS+1):
    
    print(f'.." store_history=True silent=False shell_futures=True cell_id=9000ce4e-7134-4c56-b95b-926cb5194b18> result=None>,),kwargs {}:


TypeError: _pause_backend() takes 1 positional argument but 2 were given

## 在测试集上评价

In [55]:
# 载入最佳模型作为当前模型
model = torch.load('checkpoint/best-{:.3f}.pth'.format(best_test_accuracy))

In [56]:
model.eval()
print(evaluate_testset())

{'epoch': 100, 'test_loss': 1.0801249, 'test_accuracy': 0.8035714285714286, 'test_precision': 0.7456140350877194, 'test_recall': 0.7233333333333334, 'test_f1-score': 0.7318181818181818}
